In [ ]:
import pandas as pd
from pathlib import Path
import plotly.graph_objs as go
import os

TUTORIAL_DIR = Path(os.getcwd()).as_posix()

# Calibration of Building Energy Simulation — Tutorial

This notebook serves as a guided tutorial for understanding and applying a systematic calibration methodology to building energy models. The approach is based on the FiabGP project and is implemented using the open-source Python module [energytool](https://github.com/BuildingEnergySimulationTools/energytool), which interfaces with EnergyPlus simulations.

We will go step-by-step through the key stages of a real-world calibration process, from raw measurements to final simulation error analysis, including:

- **Understanding the role of calibration** in building performance modeling
- **Assessing the initial simulation error** using standard metrics such as NMBE and CV(RMSE)
- **Adjusting the model using measured data** (temperature, weather, usage)
- **Identifying key uncertain parameters** through global sensitivity analysis (Morris method)
- **Performing model calibration** using optimization algorithms
- **Interpreting the results** to assess model validity and limitations

---

The methodology is applied to a real 7-story multifamily building located in *Pessac*, France. As part of a large-scale renovation project, a measurement campaign was carried out from **September 2021 to July 2022** in four monitored apartments.

## Project context:

- **Thermal envelope renovation** using different types of insulation (resol, rockwool 160–300 mm)
- **Replacement of windows** (double glazing)
- **Mechanical ventilation system (VMC)** installed
- **Infiltration tests** showing improved airtightness
- **Sensors installed** to measure:
  - Indoor temperature (living room & bedroom)
  - Gas consumption
  - Heating power delivered (space heating & DHW)

---


|             Figure 1: Building model sketch              |                 Figure 2: Building measurement overview                  |
|:-------------------------------------------------------------:|:---------------------------------------------------------------:|
| <img height="300" src="resources/building.png"/> | <img src="resources/measurements.png"  height="300"> |

## Load and set model

### Boundary file

In [ ]:
boundaries = pd.read_csv(
    Path(TUTORIAL_DIR) / "resources/mean_temp_heating.csv",
    index_col=0,
    parse_dates=True,
)

In [ ]:
heating_columns = [
    "2D_Heating_power_[Wh]",
    "4D_Heating_power_[Wh]",
    "4G_Heating_power_[Wh]",
    "7D_Heating_power_[Wh]",
]

# Calculate daily heating consumption in kWh
daily_cons_measure_kWh = (
    boundaries[heating_columns].resample("D").sum().sum(axis=1) / 1000
)
daily_cons_measure_kWh.name = "Total_heating_consumption_[kWh]"

# Calculate hourly heating consumption in kWh
hourly_cons_measure_kWh = boundaries[heating_columns].sum(axis=1) / 1000
hourly_cons_measure_kWh.name = "Total_heating_energy_[kWh]"

### Load idf and add systems

In [ ]:
from energytool.building import Building, SimuOpt
from energytool.system import HeaterSimple, AirHandlingUnit, Sensor, ZoneThermostat

In [ ]:
building = Building(idf_path=Path(TUTORIAL_DIR) / "resources/model_v0_store_op.idf")

Define zone for heating and temperature sensors

In [ ]:
thermal_zones = ["R4:4G", "R4:4D", "R2:2D", "R7:7D"]

In [ ]:
building.add_system(HeaterSimple(name="gaz_boiler", zones=thermal_zones, cop=1))

building.add_system(
    AirHandlingUnit(
        name="Extraction_fan",
        zones=thermal_zones,
        ach=0.7,
        fan_energy_coefficient=0.23,  # Wh/m3
        heat_recovery_efficiency=False,
    )
)


building.add_system(
    Sensor(
        name="Hobos", variables="Zone Operative Temperature", key_values=thermal_zones
    )
)

building.add_system(
    Sensor(
        name="meteo", variables="Site Outdoor Air Drybulb Temperature", key_values="*"
    )
)

# Step 1 — Initial Simulation and Error Evaluation

In this step, we run the initial EnergyPlus simulation of the building using the default modeling assumptions (e.g., standard weather file, standard indoor temperature schedules, and system operation).

The outputs are compared to the measured data over the heating season, using key error metrics recommended by **ASHRAE Guideline 14** and **IPMVP**:

- **NMBE** (Normalized Mean Bias Error): assesses bias in predictions  
- **CV(RMSE)** (Coefficient of Variation of the Root Mean Squared Error): evaluates the spread of the prediction errors  

Both metrics are calculated on:
- **Monthly** aggregated heating energy
- **Hourly** heating demand profile

We also visually compare:
- The **simulated heating energy** vs. **measured boiler output**  
- The **simulated indoor temperatures** vs. **measured temperatures**    for the 4D apartment (reference unit)


In [ ]:
from copy import deepcopy

In [ ]:
sim_opt = {
    SimuOpt.EPW_FILE.value: Path(TUTORIAL_DIR) / r"FRA_Bordeaux.075100_IWEC.epw",
    SimuOpt.OUTPUTS.value: "SYSTEM|SENSOR",
    SimuOpt.START.value: "2021-07-01 00:00",
    SimuOpt.STOP.value: "2022-06-30 23:00",
    SimuOpt.VERBOSE.value: "v",
}

In [ ]:
init_sim = building.simulate(parameter_dict=None, simulation_options=sim_opt)

In [ ]:
sim_heating_kWh = (init_sim["HEATING_Energy_[J]"] / 3.6e6).loc["2021-10":"2022-04"]

to_plot = pd.concat(
    [
        daily_cons_measure_kWh,
        sim_heating_kWh.resample("D").sum(),
    ],
    axis=1,
)

fig = go.Figure()
for dat in to_plot.columns:
    fig.add_trace(go.Bar(x=init_sim.index, y=to_plot[dat], name=dat))

fig.update_layout(dict(title="Heating Energy [kWh]", yaxis_title="Energy [kWh]"))

fig.show()

In [ ]:
to_plot = pd.concat(
    [boundaries["T_4D"], init_sim["R4:4D_Zone Operative Temperature"]], axis=1
)

fig = go.Figure()
for dat in to_plot.columns:
    fig.add_trace(go.Scatter(x=init_sim.index, y=to_plot[dat], name=dat))

fig.update_layout(
    dict(title="Appartment 4D - Operative temperature", yaxis_title="Temperature [°C]")
)

fig.show()

In [ ]:
from sklearn.metrics import r2_score
from corrai.metrics import cv_rmse, nmbe

In [ ]:
measured_heating_kWh = hourly_cons_measure_kWh.loc["2021-10":"2022-04"]

# --- Hourly results---
sim_hourly = sim_heating_kWh.resample("h").sum()
measured_hourly = measured_heating_kWh.resample("h").sum()

nmbe_hourly = nmbe(sim_hourly, measured_hourly)
cv_rmse_hourly = cv_rmse(sim_hourly, measured_hourly)
r2_hourly = r2_score(sim_hourly, measured_hourly)

# --- Monthly results ---
sim_monthly = sim_heating_kWh.resample("ME").sum()
measured_monthly = measured_heating_kWh.resample("ME").sum()

nmbe_monthly = nmbe(sim_monthly, measured_monthly)
cv_rmse_monthly = cv_rmse(sim_monthly, measured_monthly)
r2_monthly = r2_score(sim_monthly, measured_monthly)

In [ ]:
print("Hourly comparison:")
print(f"  NMBE:    {nmbe_hourly / 100:.2%}")
print(f"  CV(RMSE): {cv_rmse_hourly / 100:.2%}")
print(f"  R²:      {r2_hourly:.3f}")

print("\nMonthly comparison:")
print(f"  NMBE:    {nmbe_monthly / 100:.2%}")
print(f"  CV(RMSE): {cv_rmse_monthly / 100:.2%}")
print(f"  R²:      {r2_monthly:.3f}")

## Conclusion on the intial model performance

The initial simulation results show a **very poor agreement** with the measured data.

Although the **seasonal trend** of heating demand is somewhat reproduced, the **magnitude of simulated heating needs is consistently underestimated by about 50%**.

In particular:
- Simulated indoor temperatures during winter are **higher than those actually measured**, despite using a heating setpoint of 22°C.
- Heating needs are much **lower than expected**, despite colder simulated temperatures.

These results indicate that the model, in its current state, **cannot be used for performance assessment or comparison with measured data** from the 2021–2022 monitoring campaign.

This is not unexpected: both **weather data** and **indoor temperature assumptions** are too far from actual conditions. While the model may qualitatively reflect the building’s behavior, its predictive capability is insufficient.

> In the next step, we will **replace key assumptions** (notably weather and indoor temperature schedules) with actual measurements, to test whether this adjustment alone reduces the prediction error.


# Step 2 — Adjustment Using Measured Inputs

To improve the model’s realism, we now replace default inputs with **measured data** from the monitoring campaign:

- A **custom historical weather file** (based on ERA5 reanalysis) replaces the standard TRY file.
- The **measured indoor temperatures** for each monitored apartment are used as **operative temperature setpoints** in the model.

> These adjustments should help isolate the impact of model assumptions (e.g., envelope performance, internal gains) from errors due to incorrect boundary conditions.

We will simulate the same period again and evaluate the new errors compared to measurements.


In [ ]:
building.add_system(
    ZoneThermostat(
        name="2D_thermos",
        zones="R2:2D".upper(),
        heating_time_series=boundaries.T_2D,
        add_schedules_output_variables=True,
        overwrite_heating_availability=True,
    )
)

building.add_system(
    ZoneThermostat(
        name="4D_thermos",
        zones="R4:4D".upper(),
        heating_time_series=boundaries.T_4D,
        add_schedules_output_variables=True,
        overwrite_heating_availability=True,
    )
)

building.add_system(
    ZoneThermostat(
        name="4G_thermos",
        zones="R4:4G".upper(),
        heating_time_series=boundaries.T_4G,
        add_schedules_output_variables=True,
        overwrite_heating_availability=True,
    )
)

building.add_system(
    ZoneThermostat(
        name="7D_thermos",
        zones="R7:7D".upper(),
        heating_time_series=boundaries.T_7D,
        add_schedules_output_variables=True,
        overwrite_heating_availability=True,
    )
)

In [ ]:
sim_opt = {
    SimuOpt.EPW_FILE.value: Path(TUTORIAL_DIR) / r"pessac_2021_07_2022_06.epw",
    SimuOpt.OUTPUTS.value: "SYSTEM|SENSOR",
    SimuOpt.START.value: "2021-07-01 00:00",
    SimuOpt.STOP.value: "2022-06-30 23:00",
    SimuOpt.VERBOSE.value: "v",
}

In [ ]:
adjusted_sim = building.simulate(parameter_dict=None, simulation_options=sim_opt)

In [ ]:
sim_heating_kWh = (adjusted_sim["HEATING_Energy_[J]"] / 3.6e6).loc["2021-10":"2022-04"]

In [ ]:
to_plot = pd.concat(
    [daily_cons_measure_kWh, sim_heating_kWh.resample("1D").sum()], axis=1
)

fig = go.Figure()
for dat in to_plot.columns:
    fig.add_trace(go.Bar(x=adjusted_sim.index, y=to_plot[dat], name=dat))

fig.update_layout(dict(title="Heating Energy [kWh]", yaxis_title="Energy [kWh]"))

fig.show()

In [ ]:
to_plot = pd.concat(
    [boundaries["T_4D"], adjusted_sim["R4:4D_Zone Operative Temperature"]], axis=1
)

fig = go.Figure()
for dat in to_plot.columns:
    fig.add_trace(go.Scatter(x=adjusted_sim.index, y=to_plot[dat], name=dat))

fig.update_layout(
    dict(title="Appartment 4D - Operative temperature", yaxis_title="Temperature [°C]")
)

fig.show()

In [ ]:
measured_heating_kWh = hourly_cons_measure_kWh.loc["2021-10":"2022-04"]

# --- Hourly results---
sim_hourly = sim_heating_kWh.resample("h").sum()
measured_hourly = measured_heating_kWh.resample("h").sum()

nmbe_hourly = nmbe(sim_hourly, measured_hourly)
cv_rmse_hourly = cv_rmse(sim_hourly, measured_hourly)
r2_hourly = r2_score(sim_hourly, measured_hourly)

# --- Monthly results ---
sim_monthly = sim_heating_kWh.resample("ME").sum()
measured_monthly = measured_heating_kWh.resample("ME").sum()

nmbe_monthly = nmbe(sim_monthly, measured_monthly)
cv_rmse_monthly = cv_rmse(sim_monthly, measured_monthly)
r2_monthly = r2_score(sim_monthly, measured_monthly)

In [ ]:
print("Hourly comparison:")
print(f"  NMBE:    {nmbe_hourly / 100:.2%}")
print(f"  CV(RMSE): {cv_rmse_hourly / 100:.2%}")
print(f"  R²:      {r2_hourly:.3f}")

print("\nMonthly comparison:")
print(f"  NMBE:    {nmbe_monthly / 100:.2%}")
print(f"  CV(RMSE): {cv_rmse_monthly / 100:.2%}")
print(f"  R²:      {r2_monthly:.3f}")

## Conclusion on the adjusted simulation

The adjusted simulation confirms the concerns raised in Step 1.

By using the measured indoor temperatures as operative temperature setpoints, we expected the simulated and measured temperatures to closely align—especially during winter. However, the simulated temperatures often exceed the measured ones, suggesting that:

- **Envelope heat losses (transmission and air exchange) are underestimated**, and/or
- **Internal and solar gains are overestimated** in the model.

In terms of energy demand:

- The **heating needs remain significantly underestimated** (NMBE around -60%), even with accurate boundary conditions.
- The **CV(RMSE)** slightly improves on an hourly basis, indicating a modest improvement in capturing day-to-day dynamics.
- However, the overall magnitude of the error confirms that the model still lacks realism in its current state.

> These results highlight the need to move beyond simply replacing inputs. The next step consists of systematically identifying the model assumptions that most influence the simulation error.


# Step 3 — Sensitivity Analysis to Identify Key Uncertainties

In this step, we aim to understand which model assumptions have the greatest impact on the mismatch between simulation and measurement.

We use the **Morris method** (a global sensitivity analysis technique) to evaluate how uncertainty in selected parameters propagates to simulation outputs. See Chapter 2 of energytool for more info on sensitivity analyses.
Specifically, we focus on the **RMSE between simulated and measured heating demand**, during the winter period (January–February 2022), as the main calibration objective.

The parameters analyzed include:

- Glazing thermal conductivity and solar gain factor
- Mechanical ventilation air change rates
- Thermal conductivity of insulation materials
- Infiltration rates
- Internal gains (equipment loads)
- Thermal capacity of materials

Each parameter is varied within a realistic uncertainty range, and a set of simulations is run to explore their effects.

> This analysis helps prioritize which parameters should be targeted in the calibration phase.


In [ ]:
from corrai.base.parameter import Parameter

In [ ]:
uncertain_param_List = [
    {
        Parameter.NAME: "UFactor",
        Parameter.INTERVAL: [2.5, 5],
        Parameter.TYPE: "Real",
    },
    {
        Parameter.NAME: "SHGC",
        Parameter.INTERVAL: [0.2, 0.65],
        Parameter.TYPE: "Real",
    },
    {
        Parameter.NAME: "ACH",
        Parameter.INTERVAL: [0.3, 1.5],
        Parameter.TYPE: "Real",
    },
    {
        Parameter.NAME: "idf.Material.Concrete Block (Heavyweight)_.2.Specific_Heat",
        Parameter.INTERVAL: [100, 840 + 1.5 * 840],
        Parameter.TYPE: "Real",
    },
    {
        Parameter.NAME: "idf.Material.Rock wool - at 10C degrees_.16.Conductivity",
        Parameter.INTERVAL: [0.028, 0.05],
        Parameter.TYPE: "Real",
    },
    {
        Parameter.NAME: "Infilt_appart",
        Parameter.INTERVAL: [0.004721 - 0.3 * 0.004721, 0.004721 + 0.3 * 0.004721],
        Parameter.TYPE: "Real",
    },
]

In [ ]:
uncertain_param_List, parameter_dict

In energy modeling, some parameters in the simulation affect **multiple objects or fields** simultaneously.

For example:
- A single parameter like `"Insulation_Therm_Conductivity"` might need to update the **thermal conductivity** of several different insulation materials in the IDF file.
- A user-defined parameter like `"Infilt_appart"` could refer to **air infiltration rates** in multiple apartment zones.

To handle this, we use a flexible system called `param_mappings` which **maps a user-defined parameter name** (used in sensitivity analysis or optimization) to one or more actual simulation inputs (IDF paths, system variables, etc.).

It allows us to:

- Abstract complex model modifications into a **single high-level parameter**
- Apply the same value to **multiple targets** in the model
- Handle **categorical parameters** that translate into multiple changes (e.g., control strategies or weather files)

In [ ]:
param_mappings = {
    "Infilt_appart": [
        "idf.ZoneInfiltration:DesignFlowRate.R4:4G Infiltration.Design_Flow_Rate",
        "idf.ZoneInfiltration:DesignFlowRate.R4:4D Infiltration.Design_Flow_Rate",
        "idf.ZoneInfiltration:DesignFlowRate.R2:2D Infiltration.Design_Flow_Rate",
        "idf.ZoneInfiltration:DesignFlowRate.R7:7D Infiltration.Design_Flow_Rate",
    ],
    "UFactor": [
        "idf.WindowMaterial:SimpleGlazingSystem.Simple Vitrage_loggia - 1001.UFactor",
        "idf.WindowMaterial:SimpleGlazingSystem.Simple Vitrage_ext - 1002.UFactor",
        "idf.WindowMaterial:SimpleGlazingSystem.Simple Vitrage_ext - 1003.UFactor",
    ],
    "SHGC": [
        "idf.WindowMaterial:SimpleGlazingSystem.Simple Vitrage_loggia - 1001.Solar_Heat_Gain_Coefficient",
        "idf.WindowMaterial:SimpleGlazingSystem.Simple Vitrage_ext - 1002.Solar_Heat_Gain_Coefficient",
        "idf.WindowMaterial:SimpleGlazingSystem.Simple Vitrage_ext - 1003.Solar_Heat_Gain_Coefficient",
    ],
    "ACH": [
        "idf.DesignSpecification:OutdoorAir.R4:4G.Outdoor_Air_Flow_Air_Changes_per_Hour",
        "idf.DesignSpecification:OutdoorAir.R4:4D.Outdoor_Air_Flow_Air_Changes_per_Hour",
        "idf.DesignSpecification:OutdoorAir.R2:2D.Outdoor_Air_Flow_Air_Changes_per_Hour",
        "idf.DesignSpecification:OutdoorAir.R7:7D.Outdoor_Air_Flow_Air_Changes_per_Hour",
    ],
}

The following cell show an example of a resulting dictionnary of parameters values using this parameter mapping: 

In [ ]:
from corrai.learning.sampling import expand_parameter_dict

param_init_values = {
    param[Parameter.NAME]: param[Parameter.INTERVAL][0]
    for param in uncertain_param_List
}
expand_parameter_dict(param_init_values, param_mappings)

In [ ]:
from corrai.sensitivity import SAnalysis, Method

In [ ]:
sa_analysis = SAnalysis(
    parameters_list=uncertain_param_List,
    method=Method.MORRIS,
)

In [ ]:
sa_analysis.draw_sample(n=5)

In [ ]:
len(sa_analysis.sample)

In [ ]:
sim_opt_SA = {
    SimuOpt.EPW_FILE.value: Path(TUTORIAL_DIR) / r"pessac_2021_07_2022_06.epw",
    SimuOpt.OUTPUTS.value: "SYSTEM|SENSOR",
    SimuOpt.START.value: "2022-01-01 00:00",
    SimuOpt.STOP.value: "2022-02-05 23:00",
    SimuOpt.VERBOSE.value: "v",
}

In [ ]:
sa_analysis.evaluate(
    model=building,
    simulation_options=sim_opt_SA,
    simulate_kwargs={"param_mapping": param_mappings},
)

In [ ]:
from corrai.metrics import cv_rmse, nmbe
import numpy as np

sa_analysis.analyze(
    indicator="HEATING_Energy_[J]",
    agg_method=np.sum,
)

In [ ]:
from corrai.sensitivity import plot_morris_scatter

sa_analysis.calculate_sensitivity_indicators()
plot_morris_scatter(
    salib_res=sa_analysis.sensitivity_results,
    title="Elementary effects",
    unit="J",
    autosize=True,
)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
for sim in sa_analysis.sample_results:
    sim[2]["HEATING_Energy_kWh"] = sim[2]["HEATING_Energy_[J]"] / 3600 / 1000

In [ ]:
sa_analysis.analyze(
    indicator="HEATING_Energy_kWh",
    reference_df=hourly_cons_measure_kWh.loc["2022-01-01 00:00":"2022-02-05 23:00"],
    agg_method=mean_squared_error,
)

In [ ]:
plot_morris_scatter(
    sa_analysis.sensitivity_results, title="Elementary effects", unit="J"
)

## Morris Sensitivity Analysis results interpretation

The Morris method provides insight into how variations in model parameters affect simulation outputs.

Two different outputs were analyzed:
- **Total simulated heating demand** over the winter period (January–February 2022)
- **RMSE between simulated and measured heating demand**, used as a calibration objective


### Simulated Heating Demand

The first sensitivity analysis measures how each parameter influences the total predicted heating needs.

- Most parameters exhibit a **linear and additive influence**, as indicated by relatively high μ* (mean absolute effect) and low σ (standard deviation).
- Key influential parameters include:
  - **Glazing conductivity**
  - **Solar heat gain coefficient (SHGC)**
  - **Ventilation air change rate**
  - **Envelope insulation conductivity**
  - **Internal gains from equipment**

- Parameters such as **loggia airflow rate** and **thermal capacity of materials** have a minimal impact on the total heating needs.

### RMSE Between Simulation and Measurement

The second analysis focuses on the **error** between the model prediction and measured heating data.
- Results show a **stronger non-linearity** and higher interaction effects (high σ values), indicating **complex or non-monotonic relationships**, with similar ranking of parameter importance.


# Step 4 — Model Calibration

Following the sensitivity analysis, we now proceed to calibrate the model by identifying the combination of uncertain parameter values that best minimizes the error between simulation and measurement.

The calibration process aims to reduce the discrepancy between:
- **Simulated heating demand**, and
- **Measured heating demand**, aggregated over the winter period (November to April)

---
Due to the **non-linear** and potentially **non-monotonic** relationship between parameters and model error (as shown by the Morris analysis), the calibration problem is inherently difficult. Traditional gradient-based optimizers are not well-suited for such problems.

We therefore use a **genetic algorithm** — a population-based, stochastic optimization method known for its robustness in complex search spaces.

However, running thousands of EnergyPlus simulations can be computationally expensive and time-consuming. To address this, we employ a **surrogate model** (also known as a **metamodel**) that approximates the relationship between input parameters and the RMSE error.

The surrogate model is trained on a sample of EnergyPlus simulations (e.g., from the Morris experiment), and then used to:

- Quickly estimate the error for new parameter combinations
- Guide the optimization algorithm toward promising regions of the parameter space
- Reduce the number of expensive full simulations required

---

`ModelSampler` from corrai.learning can be used first, to: 

- Generate parameter samples using a **space-filling design** (e.g., Latin Hypercube)
- Interface with the simulation model to **run the simulations automatically**
- Store both the **sampled parameter sets** and the corresponding **simulation outputs**

This class acts as the bridge between your model and the uncertainty exploration needed for sensitivity analysis or surrogate modeling.


In [ ]:
from corrai.learning.sampling import ModelSampler

In [ ]:
sampler = ModelSampler(
    parameters=uncertain_param_List,
    model=building,
    sampling_method="LatinHypercube",
    simulation_options=sim_opt_SA,
    param_mappings=param_mappings,
)

The `ModelSampler` class provides two methods for sampling variants: `add_sample` and `draw_sample`. 

- `add_sample`: This method adds a sample of of size n. This method is useful when you want to build a specific sample incrementally, running systematically simulation for each sampel, progressively adding variants until the desired sample size is reached.

- `draw_sample`: This method draws a sample of parameters.It will only select combinations without simulating them. They can be simulated later on, using method `simulate_combinations`.

For instance, we could generate 150 simulations using: 
<code> sampler.add_sample(sample_size=150, seed=42) </code>. 

Since we simulated 70 combinations for the sensitivity analysis, let's use them instead in this tutorial. Note that 70 simulations for several parameters is not much at all.

In [ ]:
sampler.sample = sa_analysis.sample
sampler.sample_results = [sim[2] for sim in sa_analysis.sample_results]

##  Surrogate Modeling with `ModelTrainer` and `MultiModelSO`

To reduce computational cost during calibration, we use a **surrogate model** (or **metamodel**) — a fast approximation of the simulation model that maps parameter inputs to a specific output (e.g., RMSE).

Two tools are available for building and selecting surrogate models:

---

###  `ModelTrainer` — Train a Single Surrogate

The `ModelTrainer` wraps any scikit-learn-compatible regression pipeline and handles:

- Splitting training and testing data
- Fitting the model to simulation results
- Evaluating the prediction error using standard metrics:
  - `test_nmbe_score`
  - `test_cvrmse_score`

### `MultiModelSO` — Comparing and selecting the best surrogate

The `MultiModelSO` class allows you to easily compare several surrogate models and automatically select the best-performing one.

It provides:

- Training of multiple regression models at once:
- Linear, polynomial, support vector machines, random forests, neural networks, etc.
- Cross-validation for fair model comparison
- Automatic selection of the best model based on a scoring metric (e.g., RMSE)
- Optional hyperparameter tuning using grid search


In [ ]:
from corrai.learning.model_selection import ModelTrainer, MultiModelSO

Our target **y** (indicator) here is the difference between measured and simulated heating energy comsumption, and our training data set is the simulated sample:

In [ ]:
reference_sim = building.simulate(parameter_dict=None, simulation_options=sim_opt_SA)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
n = len(sampler.sample_results)

y_preds = [r["HEATING_Energy_kWh"].sum() for r in sampler.sample_results]
y_true = np.full(n, hourly_cons_measure_kWh.loc["2022-01-01":"2022-02-05"].sum())

mse_list = [mean_squared_error([true], [pred]) for true, pred in zip(y_true, y_preds)]
y = pd.Series(mse_list)

In [ ]:
x_train_df = pd.DataFrame(sampler.sample)

In [ ]:
modelSO = MultiModelSO(fine_tuning=False)
trainer = ModelTrainer(modelSO)
trainer.train(X=x_train_df, y=y)

Let's check the r-score of the best model on prediciton data:

In [ ]:
trainer.model_pipe.score(trainer.x_test, trainer.y_test)

We can now predict the heating energy on new samples :

In [ ]:
sampler.draw_sample(sample_size=1000, seed=42)
x_pred_df = pd.DataFrame(sampler.not_simulated_samples)
modelSO.predict(x_pred_df)

## Optimization Using the Surrogate Model

After training and evaluating several surrogate models, we now select the **best-performing one** to use as a fast approximation of the simulation model.

This surrogate will be used to **find the combination of parameters** that minimizes the error between the model and the measurements.

We define an **objective function** that takes a vector of input parameters and returns the **predicted RMSE** from the surrogate model.

This function is passed to `scipy.optimize.minimize`, which performs a local search for the minimum error.

- The optimization is performed in the **parameter space** defined during sampling.
- The surrogate is used instead of the full simulation to keep the process fast.
- The result is a calibrated set of parameters that minimize the model-measurement discrepancy, according to the surrogate.

In [ ]:
import numpy as np


def _objective_function(x):
    x_df = pd.DataFrame([x], columns=[p[Parameter.NAME] for p in uncertain_param_List])
    y_pred = trainer.model_pipe.predict(x_df)
    return float(np.asarray(y_pred).flatten()[0])

In [ ]:
from scipy.optimize import differential_evolution

In [ ]:
res = differential_evolution(
    _objective_function,
    bounds=[tuple(param[Parameter.INTERVAL]) for param in uncertain_param_List],
)
res

In [ ]:
parameter_names = [param[Parameter.NAME] for param in uncertain_param_List]
parameter_dict = {param_name: res.x[i] for i, param_name in enumerate(parameter_names)}

Let's simulate with this set of values for our parameters

In [ ]:
sim_calibrated = building.simulate(
    parameter_dict=parameter_dict,
    simulation_options=sim_opt,
    param_mapping=param_mappings,
)

In [ ]:
import plotly.graph_objects as go

sim_daily = sim_calibrated["HEATING_Energy_[J]"].resample("D").sum() / 3600 / 1000
ref_daily = hourly_cons_measure_kWh.resample("D").sum()

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        name="Simulation",
        mode="lines",
        x=sim_daily.index,
        y=sim_daily,
        line=dict(color="green"),
    )
)

fig.add_trace(
    go.Scatter(
        name="Measurement",
        mode="lines",
        x=ref_daily.index,
        y=ref_daily,
        line=dict(color="crimson"),
    )
)

fig.update_layout(
    title="Comparison Simulation vs Measurement",
    yaxis_title="Heating [kWh/day]",
    template="simple_white",
)

In [ ]:
sim_energy_kWh = sim_calibrated["HEATING_Energy_[J]"] / 3600 / 1000
measured_energy_kWh = hourly_cons_measure_kWh

# --- Resample ---
sim_hourly = sim_energy_kWh.resample("h").sum()
measured_hourly = measured_energy_kWh.resample("h").sum()

sim_daily = sim_energy_kWh.resample("D").sum()
measured_daily = measured_energy_kWh.resample("D").sum()

sim_monthly = sim_energy_kWh.resample("ME").sum()
measured_monthly = measured_energy_kWh.resample("ME").sum()

# --- Metrics ---
nmbe_hourly = nmbe(sim_hourly, measured_hourly)
cvrmse_hourly = cv_rmse(sim_hourly, measured_hourly)
r2_hourly = r2_score(sim_hourly, measured_hourly)

nmbe_daily = nmbe(sim_daily, measured_daily)
cvrmse_daily = cv_rmse(sim_daily, measured_daily)
r2_daily = r2_score(sim_daily, measured_daily)

nmbe_monthly = nmbe(sim_monthly, measured_monthly)
cvrmse_monthly = cv_rmse(sim_monthly, measured_monthly)
r2_monthly = r2_score(sim_monthly, measured_monthly)

In [ ]:
print("Hourly comparison:")
print(f"  NMBE:     {nmbe_hourly / 100:.2%}")
print(f"  CV(RMSE): {cvrmse_hourly / 100:.2%}")
print(f"  R²:       {r2_hourly:.3f}")

print("Daily comparison:")
print(f"  NMBE:     {nmbe_daily / 100:.2%}")
print(f"  CV(RMSE): {cvrmse_daily / 100:.2%}")
print(f"  R²:       {r2_daily:.3f}")

print("Monthly comparison:")
print(f"  NMBE:     {nmbe_monthly / 100:.2%}")
print(f"  CV(RMSE): {cvrmse_monthly / 100:.2%}")
print(f"  R²:       {r2_monthly:.3f}")

## Conclusion

The calibration process has shown some promising signs, but also highlighted clear limitations of the current model.

- The calibrated results are better centered, and on average the model **underestimates the heating demand**.
- The reduction in CV(RMSE) at the monthly scale suggests that the **overall seasonal dynamics are reasonably captured**.
- However, at the **hourly level**, the error remains significant — which undermines the usefulness of the model for detailed thermal simulation.

All optimized parameter values ended up at or near the **limits of their acceptable ranges**. This is not a good sign:
> It likely indicates that the optimization algorithm attempted to escape the defined bounds to further reduce the error — implying that the model is even more inaccurate than initially expected.

Additionally, our surrogate models were trained on a relatively **small dataset** (a few dozen simulations), which is likely insufficient given the **high dimensionality** of the parameter space.

**Next steps**

- Increase the number of simulations to better explore the parameter space.
- Re-express or simplify the model if possible.
- Consider identifying the most sensitive parameters and fixing the least influential ones.

In its current state, the model should **not** be used to explain measured data from 2021–2022 or to draw strong conclusions. Further refinement and validation are needed before it becomes a reliable predictive tool.